In [ ]:
pip install tensorflow-recommenders tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.5 MB/s eta 0:00:00


# Load data

In [ ]:
from typing import Dict, Text
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_id": tf.strings.to_number(x["movie_id"]),
    "user_id": tf.strings.to_number(x["user_id"])
})
movies = movies.map(lambda x: tf.strings.to_number(x["movie_id"]))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incomplete19GL6L/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


# 沒有優化器

In [ ]:




import tensorflow_addons as tfa
# ... [省略数据加载和预处理的代码] ...

# Build a model.
class SmallModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    # 假设每个嵌入向量的大小为 32，用户和电影的数量根据数据集确定。
    embedding_dimension = 32

    # Set up user representation.
    self.user_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1000, output_dim=embedding_dimension),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
    ])

    # Set up movie representation.
    self.item_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1700, output_dim=embedding_dimension),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
    ])

    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.item_model(features["movie_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, movie_embeddings)

# Instantiate the model.
model = SmallModel()


model.compile()
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
history_5 = model.fit(train.batch(4096), epochs=50)

# Evaluate.
evaluation_metrics_5 = model.evaluate(test.batch(4096), return_dict=True)

Epoch 1/50
20/20 [==============================] - 26s 767ms/step - factorized_top_k/top_1_categorical_accuracy: 2.3750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0014 - factorized_top_k/top_10_categorical_accuracy: 0.0032 - factorized_top_k/top_50_categorical_accuracy: 0.0174 - factorized_top_k/top_100_categorical_accuracy: 0.0335 - loss: 32409.2772 - regularization_loss: 0.0000e+00 - total_loss: 32409.2772
Epoch 2/50
20/20 [==============================] - 15s 626ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0129 - factorized_top_k/top_5_categorical_accuracy: 0.0244 - factorized_top_k/top_10_categorical_accuracy: 0.0316 - factorized_top_k/top_50_categorical_accuracy: 0.0752 - factorized_top_k/top_100_categorical_accuracy: 0.1236 - loss: 31879.5695 - regularization_loss: 0.0000e+00 - total_loss: 31879.5695
Epoch 3/50
20/20 [==============================] - 15s 647ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0041 - factorized_top_k/top_5_categorical_

# clippy

In [ ]:




import tensorflow_addons as tfa
# ... [省略数据加载和预处理的代码] ...

# Build a model.
class SmallModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    # 假设每个嵌入向量的大小为 32，用户和电影的数量根据数据集确定。
    embedding_dimension = 32

    # Set up user representation.
    self.user_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1000, output_dim=embedding_dimension),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
    ])

    # Set up movie representation.
    self.item_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1700, output_dim=embedding_dimension),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
    ])

    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.item_model(features["movie_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, movie_embeddings)

# Instantiate the model.
model = SmallModel()


model.compile(optimizer=tfrs.experimental.optimizers.ClippyAdagrad(0.01))
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
history_6 = model.fit(train.batch(4096), epochs=50)

# Evaluate.
evaluation_metrics_6 = model.evaluate(test.batch(4096), return_dict=True)

Epoch 1/50
20/20 [==============================] - 23s 865ms/step - factorized_top_k/top_1_categorical_accuracy: 1.6250e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0026 - factorized_top_k/top_10_categorical_accuracy: 0.0057 - factorized_top_k/top_50_categorical_accuracy: 0.0314 - factorized_top_k/top_100_categorical_accuracy: 0.0637 - loss: 32417.5283 - regularization_loss: 0.0000e+00 - total_loss: 32417.5283
Epoch 2/50
20/20 [==============================] - 16s 695ms/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0026 - factorized_top_k/top_10_categorical_accuracy: 0.0057 - factorized_top_k/top_50_categorical_accuracy: 0.0314 - factorized_top_k/top_100_categorical_accuracy: 0.0636 - loss: 32417.5283 - regularization_loss: 0.0000e+00 - total_loss: 32417.5283
Epoch 3/50
20/20 [==============================] - 16s 693ms/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_cate

# save csv file

In [ ]:
from google.colab import files
import pandas as pd
def calculate_average_accuracy(history):
    top_1_acc = history['factorized_top_k/top_1_categorical_accuracy']
    top_5_acc = history['factorized_top_k/top_5_categorical_accuracy']
    top_10_acc = history['factorized_top_k/top_10_categorical_accuracy']
    top_50_acc = history['factorized_top_k/top_50_categorical_accuracy']
    top_100_acc = history['factorized_top_k/top_100_categorical_accuracy']
    return [(t1 + t5 + t10 + t50 + t100) / 5 for t1, t5, t10, t50, t100 in zip(top_1_acc, top_5_acc, top_10_acc, top_50_acc, top_100_acc)]

# 计算每个历史记录的平均准确率
average_acc_5 = calculate_average_accuracy(history_5.history)
average_acc_6 = calculate_average_accuracy(history_6.history)

# 创建一个 DataFrame 来存储所有历史记录的平均准确率
average_acc_df = pd.DataFrame({
    'own': average_acc_5,
    'Adam': average_acc_6,
})
csv_path = 'categorical_accuracy_combined.csv'
average_acc_df.to_csv(csv_path, index=False)

print("CSV file saved to:", csv_path)
files.download(csv_path)
# 假设 history 和 history2 是两次训练过程的历史记录对象
loss_history5 = history_5.history['loss']
loss_history6 = history_6.history['loss']
# 将它们转换为 DataFrame
# 将它们转换为 DataFrame
df5 = pd.DataFrame({'No': loss_history5})
df6 = pd.DataFrame({'clippy': loss_history6})

# 合并两个 DataFrame
combined_df = pd.concat([df5,df6], axis=1)

# 打印合并后的 DataFrame
print(combined_df)

# 保存为 CSV 文件
csv_path='loss_history.csv'
combined_df.to_csv(csv_path, index=False)
print("CSV file saved to:", csv_path)
files.download(csv_path)

CSV file saved to: categorical_accuracy_combined.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

              No        clippy
0   16665.363281  16723.099609
1   16332.186523  16723.099609
2   16285.584961  16723.099609
3   16260.677734  16723.099609
4   16200.086914  16723.099609
5   16121.373047  16723.099609
6   16007.588867  16723.099609
7   15977.057617  16723.099609
8   15953.012695  16723.099609
9   15928.092773  16723.099609
10  15903.507812  16723.099609
11  15910.033203  16723.099609
12  15891.797852  16723.099609
13  15875.923828  16723.099609
14  15859.242188  16723.097656
15  15853.277344  16723.099609
16  15847.175781  16723.099609
17  15845.850586  16723.099609
18  15819.165039  16723.097656
19  15823.999023  16723.099609
20  15797.716797  16723.099609
21  15764.190430  16723.097656
22  15735.131836  16723.099609
23  15716.625000  16723.097656
24  15693.953125  16723.097656
25  15678.810547  16723.099609
26  15677.738281  16723.099609
27  15668.732422  16723.099609
28  15643.792969  16723.097656
29  15640.857422  16723.097656
30  15634.327148  16723.097656
31  1561

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>